In [8]:
from flask import Flask, redirect, url_for, render_template, request, session, flash
from keras.models import load_model
import pickle
from nltk.tokenize import RegexpTokenizer
import numpy as np
import heapq
from flask_sqlalchemy import SQLAlchemy
from datetime import datetime

#importing library
from transformers import BertTokenizer, TFBertModel
#defining tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#instantiating the model
model = TFBertModel.from_pretrained("bert-base-uncased")
text = "What is your name?"
#extracting features
encoded_input = tokenizer(text, return_tensors='tf')
model(encoded_input) 

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<tf.Tensor: shape=(1, 7, 768), dtype=float32, numpy=
array([[[ 0.08028892,  0.15121321, -0.2958817 , ..., -0.25193146,
          0.10468219,  0.46184197],
        [ 0.23416208, -0.01264843, -0.2747215 , ...,  0.17610902,
          0.38892975, -0.2551906 ],
        [ 0.47130805,  0.02673543, -0.17008346, ..., -0.5424892 ,
          0.14111695, -0.23771217],
        ...,
        [ 0.03444475, -0.32429618,  0.07769899, ..., -0.02780228,
          0.08623265, -0.8596272 ],
        [ 0.03843497, -0.07758588, -0.96168417, ..., -0.03568524,
          0.27894577, -0.13651596],
        [ 0.8725083 , -0.02664122, -0.5582643 , ...,  0.03990697,
         -0.62972516, -0.35996273]]], dtype=float32)>, pooler_output=<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
array([[-9.12194133e-01, -4.65130001e-01, -7.82819688e-01,
         8.21622789e-01,  3.97013754e-01, -1.70064911e-01,
         9.12612855e-01,  3.89790773e-01, -6.42275691e-0

In [9]:
from transformers import pipeline
model = pipeline('fill-mask', model='bert-base-uncased')

All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [14]:
app = Flask(__name__)
app.secret_key = "esra"
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///users.sqlite3'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///history.sqlite3'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)

class users(db.Model):
    id = db.Column(db.Integer(), primary_key = True)
    user = db.Column(db.String(100))
    
    def __init__(self,user):
        self.user = user 
        
class history(db.Model):
    id = db.Column(db.Integer(), primary_key = True)
    user_id = db.Column(db.Integer())
    date = db.Column(db.DateTime, default=datetime.utcnow)
    text = db.Column(db.String(100))
    choice = db.Column(db.String(100))
    
    def __init__(self,user_id, date, text, choice):
        self.user_id = user_id 
        self.date = date
        self.text = text 
        self.choice = choice 
        
@app.route("/", methods=["POST", "GET"])
def home():
    if request.method == "POST":
        username_form = request.form["username"]
        session["user"] = username_form
        
        found_user = users.query.filter_by(user=username_form).first()
        
        if found_user:
#            session["userID"] = found_user.id
            flash("Login Successfull!")
        else:
            usr = users(username_form)
            db.session.add(usr)
            db.session.commit()
            found_user = users.query.filter_by(user=username_form).first()
#            session["userID"] = found_user.id
        
        return redirect(url_for("main", text= "a"))
    else:
#        if "user" in session:
#            flash("Already logged in!")
        return render_template("index.html")
    

@app.route("/main/<text>", methods=["POST", "GET"])
def main(text):
    if "user" in session:
            user = session["user"]
    if request.method == "POST":
        input_text = request.form["text"]
        input_ready = input_text + " " + "[MASK]" + "."
        pred = model(input_ready)
        return redirect(url_for("results", text = input_text, username = user, suggestions1 = pred[0]['token_str'], suggestions2 = pred[1]['token_str'], suggestions3 = pred[2]['token_str'], suggestions4 = pred[3]['token_str'], suggestions5=pred[4]['token_str']))
    else:
        flash("Login Successfull!")
        return render_template("main.html", username = user, text = text)
    
@app.route("/view")
def view():
    if "user" in session:
            user = session["user"]
    return render_template("view.html", values=users.query.all(), username = user)

@app.route("/user_history")
def user_history():
    if "user" in session:
            user = session["user"]
    found_user = users.query.filter_by(user=user).first()       
    return render_template("user_history.html", values=history.query.filter_by(user_id=found_user.id).all(), username = user)

@app.route("/results/<text>/<username>/<suggestions1>/<suggestions2>/<suggestions3>/<suggestions4>/<suggestions5>", methods=["POST", "GET"])
def results(text, username, suggestions1, suggestions2, suggestions3, suggestions4, suggestions5):
    if request.method == "POST" and request.form['suggestions']:
        updated_text = text + " " + request.form['suggestions']
        
        #saving user choice in history.db
        found_user = users.query.filter_by(user=username).first()
        curr_date=datetime.now()
        history_input = history(found_user.id, curr_date, text, request.form['suggestions'])
        db.session.add(history_input)
        db.session.commit()
        
        return redirect(url_for("main", text = updated_text))
    else:
        return render_template("results.html", text = text, username = username, suggestions1 = suggestions1, suggestions2 = suggestions2, suggestions3 = suggestions3, suggestions4 = suggestions4, suggestions5=suggestions5)

if __name__=='__main__':
    db.create_all()
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Apr/2022 17:13:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Apr/2022 17:13:30] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [12/Apr/2022 17:13:30] "GET /main/a HTTP/1.1" 200 -
127.0.0.1 - - [12/Apr/2022 17:13:34] "POST /main/a HTTP/1.1" 302 -
127.0.0.1 - - [12/Apr/2022 17:13:34] "GET /results/what%20is%20your%20/Esra%20Ersoy/name/problem/purpose/answer/question HTTP/1.1" 200 -
127.0.0.1 - - [12/Apr/2022 17:13:44] "POST /results/what%20is%20your%20/Esra%20Ersoy/name/problem/purpose/answer/question HTTP/1.1" 302 -
127.0.0.1 - - [12/Apr/2022 17:13:44] "GET /main/what%20is%20your%20%20name HTTP/1.1" 200 -
127.0.0.1 - - [12/Apr/2022 17:13:50] "POST /main/what%20is%20your%20%20name HTTP/1.1" 302 -
127.0.0.1 - - [12/Apr/2022 17:13:50] "GET /results/what%20is%20your%20%20name/Esra%20Ersoy/%3F/again/now/then/here HTTP/1.1" 200 -
127.0.0.1 - - [12/Apr/2022 17:13:56] "POST /results/what%20is%20your%20%20name/Esra%20Ersoy/%3F/a

127.0.0.1 - - [12/Apr/2022 17:20:52] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [12/Apr/2022 17:20:52] "GET /main/a HTTP/1.1" 200 -
127.0.0.1 - - [12/Apr/2022 17:21:37] "GET /main/a HTTP/1.1" 200 -
127.0.0.1 - - [12/Apr/2022 17:22:07] "POST /main/a HTTP/1.1" 302 -
127.0.0.1 - - [12/Apr/2022 17:22:07] "GET /results/The%20boston%20marathon%20is%20an/Hakan%20Ozman/event/annual/example/marathon/alternative HTTP/1.1" 200 -
127.0.0.1 - - [12/Apr/2022 17:22:15] "POST /results/The%20boston%20marathon%20is%20an/Hakan%20Ozman/event/annual/example/marathon/alternative HTTP/1.1" 302 -
127.0.0.1 - - [12/Apr/2022 17:22:15] "GET /main/The%20boston%20marathon%20is%20an%20annual HTTP/1.1" 200 -
127.0.0.1 - - [12/Apr/2022 17:22:16] "POST /main/The%20boston%20marathon%20is%20an%20annual HTTP/1.1" 302 -
127.0.0.1 - - [12/Apr/2022 17:22:16] "GET /results/The%20boston%20marathon%20is%20an%20annual/Hakan%20Ozman/event/race/marathon/competition/tradition HTTP/1.1" 200 -
127.0.0.1 - - [12/Apr/2022 17:22:20] "POST /